In [ ]:
!pip install pyspark


In [1]:
# import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pyspark.sql.types as T
import numpy as np
import os


def create_spark_session():
    """Create Spark session with optimized configuration for data processing"""
    spark = SparkSession.builder \
        .appName("FraudDetection_DataAcquisition") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.adaptive.skew.enabled", "true") \
        .config("spark.sql.parquet.compression.codec", "snappy") \
        .config("spark.executor.memory", "4g") \
        .config("spark.driver.memory", "2g") \
        .config("spark.memory.offHeap.enabled", "true") \
        .config("spark.memory.offHeap.size", "2g") \
        .getOrCreate()

    spark.sparkContext.setLogLevel("WARN")
    return spark


spark = create_spark_session()
print("Spark session created successfully!")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/storage/home/tpk5410/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/storage/home/tpk5410/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
25/12/06 17:56:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark session created successfully!


In [2]:
# Data Acquisition
def load_credit_card_data(spark, filepath):
    """Load the credit card fraud dataset"""
    print("Loading credit card data...")


    schema = StructType([
        StructField("Time", DoubleType(), True),
        StructField("V1", DoubleType(), True),
        StructField("V2", DoubleType(), True),
        StructField("V3", DoubleType(), True),
        StructField("V4", DoubleType(), True),
        StructField("V5", DoubleType(), True),
        StructField("V6", DoubleType(), True),
        StructField("V7", DoubleType(), True),
        StructField("V8", DoubleType(), True),
        StructField("V9", DoubleType(), True),
        StructField("V10", DoubleType(), True),
        StructField("V11", DoubleType(), True),
        StructField("V12", DoubleType(), True),
        StructField("V13", DoubleType(), True),
        StructField("V14", DoubleType(), True),
        StructField("V15", DoubleType(), True),
        StructField("V16", DoubleType(), True),
        StructField("V17", DoubleType(), True),
        StructField("V18", DoubleType(), True),
        StructField("V19", DoubleType(), True),
        StructField("V20", DoubleType(), True),
        StructField("V21", DoubleType(), True),
        StructField("V22", DoubleType(), True),
        StructField("V23", DoubleType(), True),
        StructField("V24", DoubleType(), True),
        StructField("V25", DoubleType(), True),
        StructField("V26", DoubleType(), True),
        StructField("V27", DoubleType(), True),
        StructField("V28", DoubleType(), True),
        StructField("Amount", DoubleType(), True),
        StructField("Class", IntegerType(), True)
    ])


    df = spark.read.csv(file_path, header=True, schema=schema)

    print(f"Original dataset size: {df.count():,} rows")
    print(f"Number of columns: {len(df.columns)}")

    return df

# Load the data (replace with your actual file path)
file_path = "creditcard.csv"  # Update this path
df_original = load_credit_card_data(spark, file_path)

Loading credit card data...


Original dataset size: 284,807 rows
Number of columns: 31


In [3]:
# clean data first for synthetic scaling
def clean_dataset(df):
    """Perform data cleaning operations on raw data"""
    print("\n=== DATA CLEANING (Pre-Scaling) ===")
    
    # Remove duplicates from raw data first
    original_count = df.count()
    df_clean = df.dropDuplicates()
    print(f"Duplicates removed: {original_count - df_clean.count()}")
    
    # Ensure correct data types
    df_clean = df_clean.withColumn("Class", F.col("Class").cast(IntegerType()))
    df_clean = df_clean.withColumn("Amount", F.col("Amount").cast(DoubleType()))
    
    # Handle potential outliers in Amount
    amount_stats = df_clean.approxQuantile("Amount", [0.001, 0.5, 0.999], 0.05)
    cap_value = amount_stats[2]
    df_clean = df_clean.withColumn(
        "Amount", 
        F.when(F.col("Amount") > cap_value, cap_value).otherwise(F.col("Amount"))
    )
    return df_clean

df_cleaned = clean_dataset(df_original) 


=== DATA CLEANING (Pre-Scaling) ===


25/12/06 17:58:08 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Duplicates removed: 1081


In [4]:
# Synthetic Scaling 
def scale_dataset(df, target_rows=6000000, seed=42):
    """Scale dataset using bootstrapping"""
    print("\n=== SYNTHETIC SCALING ===")
    original_count = df.count()
    scaling_factor = target_rows / original_count
    
    print(f"Original clean size: {original_count:,}")
    print(f"Scaling factor: {scaling_factor:.2f}x")
    
    # Apply bootstrapping
    df_scaled = df.sample(withReplacement=True, fraction=scaling_factor, seed=seed)
    
    # Add row_id to make rows distinct technically (helps downstream)
    df_scaled = df_scaled.withColumn("row_id", F.monotonically_increasing_id())
    
    print(f"Scaled dataset size: {df_scaled.count():,} rows")
    return df_scaled


df_scaled = scale_dataset(df_cleaned, target_rows=6000000)

def perform_data_quality_checks(df):
    print("\n=== DATA QUALITY CHECKS ===")
    # (Your original check logic here is fine, just run it on df_scaled)
    print(f"Current Row Count: {df.count():,}")
    return df

df_final_check = perform_data_quality_checks(df_scaled)

# ensure variable consistency for next steps
df_cleaned = df_final_check


=== SYNTHETIC SCALING ===


Original clean size: 283,726
Scaling factor: 21.15x


Scaled dataset size: 5,998,034 rows

=== DATA QUALITY CHECKS ===


Current Row Count: 5,998,034


In [5]:
# Basic Feature Engineering (Foundation for Division 3)
def create_basic_features(df):
    """
    Create basic feature transformations
    While Division 3 handles advanced features, we provide foundational transformations
    """
    print("\n=== BASIC FEATURE ENGINEERING ===")

    # Log transform Amount to handle skewness
    # Adding 1 to avoid log(0) issues
    df_featured = df.withColumn("Log_Amount", F.log(F.col("Amount") + 1))

    # Create time-based features (hour of day from Time column)
    # Assuming Time is in seconds, convert to hours modulo 24
    df_featured = df_featured.withColumn("Hour_of_Day", (F.col("Time") / 3600) % 24)

    # Create amount categories for basic segmentation
    df_featured = df_featured.withColumn(
        "Amount_Category",
        F.when(F.col("Amount") < 10, "Very Low")
         .when(F.col("Amount") < 50, "Low")
         .when(F.col("Amount") < 100, "Medium")
         .when(F.col("Amount") < 500, "High")
         .otherwise("Very High")
    )

    print("Basic features created: Log_Amount, Hour_of_Day, Amount_Category")

    return df_featured

df_final = create_basic_features(df_cleaned)


=== BASIC FEATURE ENGINEERING ===
Basic features created: Log_Amount, Hour_of_Day, Amount_Category


In [6]:
# Data Validation before Storage
def validate_final_dataset(df):
    """Comprehensive validation of the final dataset"""
    print("\n=== FINAL DATASET VALIDATION ===")
    
    final_count = df.count()
    print(f"1. Final row count: {final_count:,}")
    
    # check fraud distribution
    fraud_dist = df.groupBy("Class").count().collect()
    for row in fraud_dist:
        class_type = "Fraud" if row['Class'] == 1 else "Non-Fraud"
        percentage = (row['count'] / final_count) * 100
        print(f"2. {class_type}: {row['count']:,} rows ({percentage:.4f}%)")
    
    # verify schema
    print("3. Final schema:")
    df.printSchema()
    

    print("4. Data sample:")
    df.select("Time", "V1", "V2", "Amount", "Log_Amount", "Class").show(10)
    

    print("5. Final null check:")
    
    # identify numeric columns for isnan check
    numeric_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, (DoubleType, FloatType, IntegerType, LongType))]
    

    null_exprs = []
    for c in df.columns:
        if c in numeric_cols:
            # For numeric, check both NaN and Null
            expr = F.sum(F.when(F.isnan(c) | F.col(c).isNull(), 1).otherwise(0)).alias(c)
        else:
            # For strings/others, just check Null
            expr = F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c)
        null_exprs.append(expr)
            
    null_summary = df.select(null_exprs)
    null_summary.show(vertical=True)
    
    return final_count >= 5000000  # minimum size requirement
# validate dataset
is_valid = validate_final_dataset(df_final)
print(f"\nDataset validation: {'PASSED' if is_valid else 'FAILED'}")


=== FINAL DATASET VALIDATION ===


1. Final row count: 5,998,034


2. Fraud: 10,095 rows (0.1683%)
2. Non-Fraud: 5,987,939 rows (99.8317%)
3. Final schema:
root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |--

+----+----------------+------------------+------+-----------------+-----+
|Time|              V1|                V2|Amount|       Log_Amount|Class|
+----+----------------+------------------+------+-----------------+-----+
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
|73.0|1.14818692615291|0.0858370715501502| 19.77|3.033509637888021|    0|
+----+----------------+---------------

-RECORD 0--------------
 Time            | 0   
 V1              | 0   
 V2              | 0   
 V3              | 0   
 V4              | 0   
 V5              | 0   
 V6              | 0   
 V7              | 0   
 V8              | 0   
 V9              | 0   
 V10             | 0   
 V11             | 0   
 V12             | 0   
 V13             | 0   
 V14             | 0   
 V15             | 0   
 V16             | 0   
 V17             | 0   
 V18             | 0   
 V19             | 0   
 V20             | 0   
 V21             | 0   
 V22             | 0   
 V23             | 0   
 V24             | 0   
 V25             | 0   
 V26             | 0   
 V27             | 0   
 V28             | 0   
 Amount          | 0   
 Class           | 0   
 row_id          | 0   
 Log_Amount      | 0   
 Hour_of_Day     | 0   
 Amount_Category | 0   


Dataset validation: PASSED


In [7]:
# Optimization and Storage
def save_optimized_dataset(df, output_path="cleaned_fraud_data.parquet"):
    """
    Save the optimized dataset in Parquet format

    Why Parquet?
    - Columnar storage: Like a zip file for columns, making reads much faster
    - Compression: Reduces storage space significantly
    - Predicate pushdown: Filters data before reading
    - Compatible with Spark MLlib
    """
    print(f"\n=== SAVING OPTIMIZED DATASET ===")

    # Repartition for optimal distributed processing
    # 10 partitions for better parallelism
    df_optimized = df.repartition(10)

    print(f"Number of partitions: {df_optimized.rdd.getNumPartitions()}")

    # Save as Parquet with Snappy compression
    print("Writing Parquet files...")
    df_optimized.write \
        .mode("overwrite") \
        .option("compression", "snappy") \
        .parquet(output_path)

    print(f"Dataset successfully saved to: {output_path}")

    # Verify the saved data
    verify_df = spark.read.parquet(output_path)
    saved_count = verify_df.count()
    print(f"Verified saved data: {saved_count:,} rows")

    return output_path

output_path = save_optimized_dataset(df_final)



=== SAVING OPTIMIZED DATASET ===


Number of partitions: 10
Writing Parquet files...


Dataset successfully saved to: cleaned_fraud_data.parquet
Verified saved data: 5,998,034 rows


In [10]:

# Generate Summary Report
def generate_summary_report(df_original, df_final, output_path):
    """Generate a comprehensive summary of the preprocessing pipeline"""
    print("\n" + "="*60)
    print("DATA ACQUISITION & PREPROCESSING SUMMARY REPORT")
    print("="*60)

    original_count = df_original.count()
    final_count = df_final.count()

    print(f"Original dataset size: {original_count:,} rows")
    print(f"Final dataset size:    {final_count:,} rows")
    print(f"Scaling factor:        {final_count/original_count:.2f}x")

    # Fraud statistics
    original_fraud = df_original.filter(F.col("Class") == 1).count()
    final_fraud = df_final.filter(F.col("Class") == 1).count()

    print(f"\nFraud Distribution:")
    print(f"  Original: {original_fraud:,} fraud cases ({original_fraud/original_count*100:.4f}%)")
    print(f"  Final:    {final_fraud:,} fraud cases ({final_fraud/final_count*100:.4f}%)")

    print(f"\nOutput Details:")
    print(f"  Format:    Parquet (Snappy compression)")
    print(f"  Location:  {output_path}")
    print(f"  Partitions: {df_final.rdd.getNumPartitions()}")

    print(f"\nFeatures Available for Next Division:")
    features = [col for col in df_final.columns if col not in ['row_id']]
    print(f"  Total features: {len(features)}")
    print(f"  New features: Log_Amount, Hour_of_Day, Amount_Category")

    print("\n" + "="*60)

    # generate final report
generate_summary_report(df_original, df_final, output_path)

print("\nPipeline completed successfully!")
print("\nInstructions for Division 2 (EDA):")
print("1. Load the data: df = spark.read.parquet('cleaned_fraud_data.parquet')")
print("2. Verify data: df.count(), df.printSchema()")
print("3. Begin EDA with the provided clean dataset")


# spark.stop()

spark.read.parquet("cleaned_fraud_data.parquet").count()


DATA ACQUISITION & PREPROCESSING SUMMARY REPORT


Original dataset size: 284,807 rows
Final dataset size:    5,998,034 rows
Scaling factor:        21.06x



Fraud Distribution:
  Original: 492 fraud cases (0.1727%)
  Final:    10,095 fraud cases (0.1683%)

Output Details:
  Format:    Parquet (Snappy compression)
  Location:  cleaned_fraud_data.parquet
  Partitions: 4

Features Available for Next Division:
  Total features: 34
  New features: Log_Amount, Hour_of_Day, Amount_Category


Pipeline completed successfully!

Instructions for Division 2 (EDA):
1. Load the data: df = spark.read.parquet('cleaned_fraud_data.parquet')
2. Verify data: df.count(), df.printSchema()
3. Begin EDA with the provided clean dataset


5998034